# ステップ09

In [1]:
abstract type AbstractFunction end

In [2]:
mutable struct Variable
    data::Union{Array{Float64,1},Nothing}
#     data::Float64
    grad::Union{Array{Float64,1},Nothing}
    creator::Union{AbstractFunction,Nothing}  # added
    Variable(data) = new(data, nothing, nothing);
end

In [3]:
function set_creator!(this::Variable, func::AbstractFunction)
    this.creator = func
end

set_creator! (generic function with 1 method)

In [4]:
function call!(this::AbstractFunction, input::Variable)
    x = input.data
    y = forward(this, x)
    output = Variable(y)
    set_creator!(output, this)  # added
    this.input = input
    this.output = output  # added
    return output
end

call! (generic function with 1 method)

Instance factory的な関数を作る

In [5]:
mutable struct Square <: AbstractFunction
    input::Union{Variable,Nothing}
    output::Union{Variable,Nothing}
    Square() = new(nothing, nothing)
end

function forward(this::Square, x::Array{Float64,1})
    y = x .^ 2
    return y
end

function backward(this::Square, gy::Array{Float64,1})
    x = this.input.data
    gx = 2 .* x .* gy
    return gx
end

function square(x::Variable)
    f = Square()
    return call!(f, x)
end

square (generic function with 1 method)

かっこわるいがexp()はBaseとかぶるのでexp_()とする

In [6]:
mutable struct Exp <: AbstractFunction
    input::Union{Variable,Nothing}
    output::Union{Variable,Nothing}
    Exp() = new(nothing, nothing)
end

function forward(this::Exp, x::Array{Float64,1})
    y = exp.(x)
    return y
end

function backward(this::Exp, gy::Array{Float64,1})
    x = this.input.data
    gx = exp.(x) .* gy
    return gx
end

function exp_(x::Variable)
    f = Exp()
    return call!(f, x)
end

exp_ (generic function with 1 method)

最初の勾配を省略できるようにする

In [12]:
function backward!(this::Variable)
    if isnothing(this.grad)
        this.grad = fill(oneunit(this.data[1]), size(this.data))  # np.ones_like()に相当する、はず（型が要素によってばらだらだと困るが）
    end
    funcs = Array{AbstractFunction, 1}()
    push!(funcs, this.creator)
    while length(funcs) > 0
        f = pop!(funcs)
        x = f.input
        y = f.output
        x.grad = backward(f, y.grad)
        
        if !isnothing(x.creator)
            push!(funcs, x.creator)
        end
    end
end

backward! (generic function with 1 method)

In [13]:
x = Variable([0.5])

Variable([0.5], nothing, nothing)

In [14]:
y = square(exp_(square(x)))

Variable([1.648721270700128], nothing, Square(Variable([1.2840254166877414], nothing, Exp(Variable([0.25], nothing, Square(Variable([0.5], nothing, nothing), Variable(#= circular reference @-2 =#))), Variable(#= circular reference @-2 =#))), Variable(#= circular reference @-2 =#)))

In [15]:
backward!(y)

In [16]:
x.grad

1-element Array{Float64,1}:
 3.297442541400256

1. square(), exp_()で変数を初期化して返すようにした
2. 最初の勾配を省略できるようにした（np.ones_likeがないので適当に代用）
3. いままでVariableに直接floatを入れていたものを、arrayを挟んで入れるように改修（あわせてelement-wiseな演算をするように変更）

いまのところ入力は1-dim arrayに決め打ちしてしまっているが、将来的には変更が必要になるはず……